# Experiment 002: Pre-optimized Starting Point

This experiment uses the pre-optimized santa-2025.csv (score ~70.68) as starting point.

Pipeline:
1. Score the pre-optimized submission
2. Run bbox3 optimizer with extended parameters
3. Apply fix_direction post-processing
4. Validate and save submission

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.strtree import STRtree
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar
import os
import subprocess
import shutil

getcontext().prec = 30

# Define the ChristmasTree class
class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (float(0), float(tip_y)),
            (float(top_w / 2), float(tier_1_y)),
            (float(top_w / 4), float(tier_1_y)),
            (float(mid_w / 2), float(tier_2_y)),
            (float(mid_w / 4), float(tier_2_y)),
            (float(base_w / 2), float(base_y)),
            (float(trunk_w / 2), float(base_y)),
            (float(trunk_w / 2), float(trunk_bottom_y)),
            (float(-trunk_w / 2), float(trunk_bottom_y)),
            (float(-trunk_w / 2), float(base_y)),
            (float(-base_w / 2), float(base_y)),
            (float(-mid_w / 4), float(tier_2_y)),
            (float(-mid_w / 2), float(tier_2_y)),
            (float(-top_w / 4), float(tier_1_y)),
            (float(-top_w / 2), float(tier_1_y)),
        ])

        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=float(self.center_x), yoff=float(self.center_y))

print("ChristmasTree class defined")

ChristmasTree class defined


In [2]:
def load_trees_for_n(df, n):
    prefix = f"{n:03d}_"
    subset = df[df['id'].str.startswith(prefix)]
    trees = []
    for _, row in subset.iterrows():
        x = str(row['x']).lstrip('s')
        y = str(row['y']).lstrip('s')
        deg = str(row['deg']).lstrip('s')
        trees.append(ChristmasTree(x, y, deg))
    return trees

def has_overlap(trees):
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx != i:
                if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                    intersection = poly.intersection(polygons[idx])
                    if intersection.area > 1e-12:
                        return True
    return False

def get_bounding_box_side(trees):
    if not trees:
        return 0
    all_coords = []
    for tree in trees:
        coords = np.array(tree.polygon.exterior.coords)
        all_coords.append(coords)
    all_coords = np.vstack(all_coords)
    x_range = all_coords[:, 0].max() - all_coords[:, 0].min()
    y_range = all_coords[:, 1].max() - all_coords[:, 1].min()
    return max(x_range, y_range)

def score_submission(df, max_n=200):
    total_score = 0
    overlaps = []
    for n in range(1, max_n + 1):
        trees = load_trees_for_n(df, n)
        if len(trees) != n:
            print(f"Warning: n={n} has {len(trees)} trees instead of {n}")
            continue
        if has_overlap(trees):
            overlaps.append(n)
        side = get_bounding_box_side(trees)
        score_n = (side ** 2) / n
        total_score += score_n
    return total_score, overlaps

print("Scoring functions defined")

Scoring functions defined


In [3]:
# Load and score the pre-optimized submission
df_preopt = pd.read_csv('/home/code/preoptimized/santa-2025.csv')
print(f"Pre-optimized submission shape: {df_preopt.shape}")
print(df_preopt.head())

print("\nScoring pre-optimized submission...")
preopt_score, preopt_overlaps = score_submission(df_preopt)
print(f"Pre-optimized score: {preopt_score:.6f}")
print(f"Overlapping configurations: {preopt_overlaps}")

Pre-optimized submission shape: (20100, 4)
      id                       x                       y  \
0  001_0    s-48.196086194214246     s58.770984615214225   
1  002_0   s0.154097069621355887  s-0.038540742694794648   
2  002_1  s-0.154097069621372845  s-0.561459257305224058   
3  003_0      s1.123655816140301      s0.781101815992563   
4  003_1       s1.23405569584216      s1.275999500663759   

                       deg  
0                    s45.0  
1  s203.629377730656841550  
2   s23.629377730656791812  
3        s111.125132292893  
4         s66.370622269343  

Scoring pre-optimized submission...


Pre-optimized score: 70.676102
Overlapping configurations: []


In [4]:
# Also check the ensemble.csv which might be better
df_ensemble = pd.read_csv('/home/code/preoptimized/ensemble.csv')
print(f"Ensemble submission shape: {df_ensemble.shape}")

print("\nScoring ensemble submission...")
ensemble_score, ensemble_overlaps = score_submission(df_ensemble)
print(f"Ensemble score: {ensemble_score:.6f}")
print(f"Overlapping configurations: {ensemble_overlaps}")

Ensemble submission shape: (20100, 4)

Scoring ensemble submission...


Ensemble score: 70.676102
Overlapping configurations: []


In [5]:
# Use the best starting point
if ensemble_score < preopt_score and len(ensemble_overlaps) == 0:
    df_best = df_ensemble
    best_score = ensemble_score
    print(f"Using ensemble.csv as starting point (score: {best_score:.6f})")
else:
    df_best = df_preopt
    best_score = preopt_score
    print(f"Using santa-2025.csv as starting point (score: {best_score:.6f})")

# Copy to working directory
df_best.to_csv('submission.csv', index=False)
print("Saved starting point as submission.csv")

Using santa-2025.csv as starting point (score: 70.676102)
Saved starting point as submission.csv


In [6]:
# Check if bbox3 binary works
import os
os.chdir('/home/code/experiments/002_preoptimized')

# Copy bbox3 binary
shutil.copy('/home/code/preoptimized/bbox3', './bbox3')
os.chmod('./bbox3', 0o755)

# Test run with small parameters
print("Testing bbox3 binary...")
result = subprocess.run(['./bbox3', '-n', '100', '-r', '2'], capture_output=True, text=True, timeout=60)
print(result.stdout)
if result.stderr:
    print("Errors:", result.stderr)

Testing bbox3 binary...

Errors: ./bbox3: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.34' not found (required by ./bbox3)



In [ ]:
# Run the compiled bbox3 optimizer\nprint(\"Running bbox3 optimizer with -n 10000 -r 64...\")\nresult = subprocess.run(\n    ['./bbox3', '-n', '10000', '-r', '64'],\n    capture_output=True, text=True, timeout=3600\n)\nprint(result.stdout)\nif result.stderr:\n    print(\"Errors:\", result.stderr)

In [ ]:
# Run extended optimization
print("Running extended bbox3 optimization...")
print("Parameters: -n 10000 -r 64")

result = subprocess.run(
    ['./bbox3', '-n', '10000', '-r', '64'],
    capture_output=True, text=True, timeout=3600
)
print(result.stdout)
if result.stderr:
    print("Errors:", result.stderr)

In [ ]:
# Implement fix_direction optimization
def calculate_bbox_side_at_angle(angle, hull_points):
    theta = np.radians(angle)
    cos_t, sin_t = np.cos(theta), np.sin(theta)
    rotated = hull_points @ np.array([[cos_t, -sin_t], [sin_t, cos_t]]).T
    x_range = rotated[:, 0].max() - rotated[:, 0].min()
    y_range = rotated[:, 1].max() - rotated[:, 1].min()
    return max(x_range, y_range)

def optimize_rotation_for_trees(trees):
    all_points = []
    for tree in trees:
        coords = np.array(tree.polygon.exterior.coords)
        all_points.extend(coords.tolist())
    points_np = np.array(all_points)
    
    if len(points_np) < 3:
        return 0, get_bounding_box_side(trees)
    
    try:
        hull = ConvexHull(points_np)
        hull_points = points_np[hull.vertices]
    except:
        hull_points = points_np
    
    res = minimize_scalar(
        lambda a: calculate_bbox_side_at_angle(a, hull_points),
        bounds=(0.001, 89.999), method='bounded'
    )
    return res.x, res.fun

def apply_rotation_to_trees(trees, angle):
    theta = np.radians(angle)
    cos_t, sin_t = np.cos(theta), np.sin(theta)
    
    new_coords = []
    for tree in trees:
        x = float(tree.center_x)
        y = float(tree.center_y)
        new_x = x * cos_t - y * sin_t
        new_y = x * sin_t + y * cos_t
        new_angle = float(tree.angle) + angle
        new_coords.append((new_x, new_y, new_angle))
    return new_coords

def fix_direction_submission(df, max_n=200):
    new_rows = []
    improvements = 0
    total_improvement = 0
    
    for n in range(1, max_n + 1):
        trees = load_trees_for_n(df, n)
        if len(trees) != n:
            continue
        
        original_side = get_bounding_box_side(trees)
        opt_angle, opt_side = optimize_rotation_for_trees(trees)
        
        if opt_side < original_side - 1e-9:
            new_coords = apply_rotation_to_trees(trees, opt_angle)
            improvements += 1
            total_improvement += (original_side ** 2 - opt_side ** 2) / n
        else:
            new_coords = [(float(t.center_x), float(t.center_y), float(t.angle)) for t in trees]
        
        for i, (x, y, deg) in enumerate(new_coords):
            new_rows.append({
                'id': f"{n:03d}_{i}",
                'x': f"s{x}",
                'y': f"s{y}",
                'deg': f"s{deg}"
            })
    
    print(f"Improved {improvements} configurations")
    print(f"Total score improvement: {total_improvement:.6f}")
    return pd.DataFrame(new_rows)

print("fix_direction functions defined")

In [ ]:
# Load optimized submission and apply fix_direction
df_optimized = pd.read_csv('submission.csv')
print("Applying fix_direction...")
df_fixed = fix_direction_submission(df_optimized)

# Score the fixed submission
fixed_score, fixed_overlaps = score_submission(df_fixed)
print(f"\nFixed submission score: {fixed_score:.6f}")
print(f"Overlapping configurations: {fixed_overlaps}")

In [ ]:
# Save final submission
os.makedirs('/home/submission', exist_ok=True)
df_fixed.to_csv('/home/submission/submission.csv', index=False)
print(f"Saved final submission to /home/submission/submission.csv")
print(f"Final score: {fixed_score:.6f}")

# Verify
df_check = pd.read_csv('/home/submission/submission.csv')
print(f"Submission shape: {df_check.shape}")
print(df_check.head())